In [10]:
import numpy as np
from numpy import linalg
import torch
from torch import nn

In [11]:
from functools import wraps
import time


def timeit(func):
    """
    time-measure decorator
    """
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [12]:
torch.cuda.is_available() 

True

In [13]:


def random_lowspec_matrix(shape):
    """
    generate random matrix with spectral radius <=1, 
    Using the fact that if every element if matrix is less than
    1/shape, then spectral radius <=1
    """
    eps = 10 ** -3
    shape_eps = shape + eps
    return -2 / shape_eps * torch.rand(shape, shape) + 1 / shape_eps
    

def compute_omega(C, eps, spec=False):
    """
    function to compute matrix omega from article (2) formula
    Input:
        C -- covatiance matrix with shape n x n
        eps -- accuracy
        spec -- if True, then calculate spectral radius before calculate power series 
    Output:
        omgea -- calculated matrix limit (if exist) with shape n x n
        
    Raise exeption, if spectral radius is greater than 1
    
    power_vector -- tensor of shape (k,n,n), where power_vector[k,:,:] = C^k
    
    in case C = diag(a_1, a_2, ..., a_n) omega = diag(c_1, ..., c_n)
    where c_i = sum k = 0 to inf a_1^(2k)
    """
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if spec:
        spectral_norm = sorted(abs(linalg.eigvals(C.cpu())))[-1]
        if spectral_norm >= 1:
            raise ValueError(f"Matrix C has spectral radius is equal {spectral_norm} greater than one")
        print(f"spectral radius is equal {spectral_norm} ")
        
    C = C.to(device)
    power_vector = torch.eye(C.shape[0]).to(device)[None,:,:]
    dif = 1
    omega_prev = torch.eye(C.shape[0]).to(device)
    while (dif > eps):
        power_vector = torch.cat((power_vector,power_vector[-1] @ C[None,:,:]), 0)
        omega = torch.sum(torch.transpose(power_vector, 2, 1) @ power_vector, dim = 0)
        dif = torch.norm(omega - omega_prev)
        omega_prev = omega
    return omega
        
        

In [14]:


@timeit
def calculate_info_storage(omega, C, eps):
    """
    input:
        omega: covariance matrix
        C: weight matrix
        eps: error
    output:
        A -- information storage
    
    M -- matrix (w/o number) from article, where zero dimension ~ k index(from article)
    lag_vector -- tensor (Omega(0), Omega(1), Omega(2) , ... , Omega(n-1))
    
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    omega_diag = torch.diagonal(omega)
    omega = omega.to(device)
    C = C.to(device)
    omega_vector = omega[None,:,:]
    power_vector = torch.eye(C.shape[0]).to(device)[None,:,:]
    diff = 1
    M_det_prev = torch.ones(C.shape[0]).to(device)
    A_prev = torch.ones(C.shape[0]).to(device)
    it = 0
    while (diff > eps):
        it += 1
        power_vector = torch.cat((power_vector,power_vector[-1] @ C[None,:,:]), 0)
        omega_vector = torch.cat((omega_vector,omega[None,:,:]), 0)
        n = omega_vector.shape[0]
        lag_vector = omega_vector @ power_vector
        
        lag_vector_concated = torch.cat((lag_vector, torch.flip(lag_vector, [0])[:-1,:,:]), 0)
        M = lag_vector_concated[None,:,:,:]

        for i in range(n):
            M = torch.cat((M ,torch.roll(lag_vector_concated, i+1, 0)[None,:,:,:]), 0)
            
        M = M[:n,:n,:,:]
        M = torch.diagonal(M, dim1=-1, dim2=-2)
        M = torch.permute(M, (2, 0, 1))
        M_det = torch.det(M)
        A = 1/2 * torch.log(torch.abs(omega_diag) * M_det / M_det_prev)
        diff = torch.norm(A - A_prev)
        M_det_prev = M_det
        A_prev = A
    print(f"total iterations: {it}")
    return A

In [15]:
C = torch.eye(3)*0.5
#C = random_lowspec_matrix(500)
omega = compute_omega(C, 0.0001)
omega

tensor([[1.3333, 0.0000, 0.0000],
        [0.0000, 1.3333, 0.0000],
        [0.0000, 0.0000, 1.3333]], device='cuda:0')

In [16]:
calculate_info_storage(omega, C, 0.01)

total iterations: 3
Function calculate_info_storage Took 0.0068 seconds


tensor([0.1438, 0.1438, 0.1438], device='cuda:0')

In [17]:
C = random_lowspec_matrix(1500)
omega = compute_omega(C, 0.0001)
omega

tensor([[ 1.0002e+00,  7.9137e-06,  1.6611e-07,  ..., -1.1928e-06,
         -5.5055e-06,  6.8026e-06],
        [ 7.9137e-06,  1.0002e+00,  8.5705e-06,  ...,  4.1680e-06,
          3.3336e-06, -7.9577e-06],
        [ 1.6611e-07,  8.5705e-06,  1.0002e+00,  ...,  2.4113e-06,
          4.7408e-06, -3.4127e-06],
        ...,
        [-1.1928e-06,  4.1680e-06,  2.4113e-06,  ...,  1.0002e+00,
          4.3697e-06, -8.6543e-07],
        [-5.5055e-06,  3.3336e-06,  4.7408e-06,  ...,  4.3697e-06,
          1.0002e+00,  2.8611e-06],
        [ 6.8026e-06, -7.9577e-06, -3.4127e-06,  ..., -8.6543e-07,
          2.8611e-06,  1.0002e+00]], device='cuda:0')

In [18]:
calculate_info_storage(omega, C, 0.001)

total iterations: 3
Function calculate_info_storage Took 0.1331 seconds


tensor([0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002], device='cuda:0')